In [2]:
import flask
print(flask.__version__)

1.1.2


In [3]:
# Check tensorflow version and GPU availability
import tensorflow as tf
print('tf version:', tf.__version__)
print('gpu:', tf.config.list_physical_devices('GPU'))

tf version: 2.10.0
gpu: []


In [4]:
IMG_DIMEN = 224
N_COLORS = 3

In [5]:
import cv2
import numpy as np

def read_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(IMG_DIMEN, IMG_DIMEN))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype('float32')/255.0
    return img

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
    plt.figure(figsize=(6, 6))
    if N_COLORS == 3:
        plt.imshow(img)
    elif N_COLORS == 1:
         plt.imshow(img, cmap='gray')
    plt.show()

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense

pretrained_model = MobileNet(
    include_top=False,
    input_shape=(IMG_DIMEN, IMG_DIMEN, N_COLORS), 
    pooling='avg', 
)

pretrained_model.trainable = False

model = Sequential()
model.add(pretrained_model)
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.load_weights('saved-model/best-model')

ValueError: Received incompatible tensor with shape (16,) when attempting to restore variable with shape (1,) and name layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE.

In [ ]:
def classify_image(img_path):
    img = read_image(img_path)
    return float(model.predict(np.array([img])).squeeze())

In [ ]:
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/classify', methods = ['POST'])
def classify_handler():
    img_path = request.form['path']
    y_pred = classify_image(img_path)
    class_name = 'malignant' if y_pred >= 0.5 else 'benign'
    prob_malignant = y_pred
    prob_benign = 1 - y_pred
    print('img_path:', img_path)
    print('y_pred:', y_pred)
    print('class_name:', class_name)
    return {
        'prob_benign': prob_benign, 
        'prob_malignant': prob_malignant, 
        'class_name': class_name
    }

app.run(port=9000)